In [1]:
import random
import string
import re
import time
from datetime import datetime
import csv
import os.path

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import clipboard

In [2]:
ACCOUNT_URL = "https://club.pokemon.com/us/pokemon-trainer-club/parents/sign-up"
MAIL_URL = "https://temp-mail.org/en/"

In [6]:
class AccountGenerator():
    def __init__(self):
        self.wd = webdriver.Firefox()
        self.user = "pmc" + datetime.now().strftime("%y%m%d%H%M%S")
        self.password = self.generate_pw()
        
    def generate_pw(self, length=20):
        pw = ""
        for i in range(length):
            base = random.choice([string.ascii_letters, string.digits, string.punctuation])
            pw += random.choice(base)
        if re.search("[a-z]", pw) and re.search("[A-Z]", pw) and re.search("[0-9!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]", pw):
            return pw
        else:
            return self.generate_pw(length)
    
    def generate(self):
        old_clipboard_content = clipboard.paste()

        self.wd.get(ACCOUNT_URL)
        time.sleep(1)

        # Confirm Age
        self.wd.find_element_by_id("id_dob").click()
        self.wd.find_element_by_xpath("/html/body/div[3]/div/section/div/div/div[1]/form/div[1]/div/div/div/div/div/div[1]/div[1]/div").click()
        self.wd.find_element_by_xpath("/html/body/div[3]/div/section/div/div/div[1]/form/div[1]/div/div/div/div/div/div[1]/div[1]/div/div/div[1]/ul/li[28]").click()
        self.wd.find_element_by_xpath("/html/body/div[3]/div/section/div/div/div[1]/form/div[1]/div/div/div/div/div/div[2]/button").click()
        time.sleep(1)
        self.wd.find_element_by_xpath("/html/body/div[3]/div/section/div/div/div[1]/form/input[2]").click()

        self.wd.find_element_by_id("id_username").send_keys(self.user)
        self.wd.find_element_by_id("id_password").send_keys(self.password)
        self.wd.find_element_by_id("id_confirm_password").send_keys(self.password)

        self.wd.execute_script("window.open('');")
        self.wd.switch_to.window(self.wd.window_handles[1])
        self.wd.get(MAIL_URL)
        while clipboard.paste() == old_clipboard_content:
            self.wd.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div[1]/form/div[2]/button").click()
        self.mail = clipboard.paste()

        self.wd.switch_to.window(self.wd.window_handles[0])

        self.wd.find_element_by_id("id_email").send_keys(self.mail)
        self.wd.find_element_by_id("id_confirm_email").send_keys(self.mail)

        self.wd.find_element_by_id("id_screen_name").send_keys(self.user)

        self.wd.find_element_by_id("id_terms").click()

        print(f"Filled information:\nUser:\t\t{self.user}\nPassword\t{self.password}\nMail\t\t{self.mail}")
        return dict(user=self.user, password=self.password, mail=self.mail)

In [12]:
ag = AccountGenerator()
acc_info = ag.generate()

if not os.path.exists(r'accounts.csv'):
    with open(r'accounts.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["Username", "Password", "Mail"])
        
with open(r'accounts.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow([acc_info['user'], acc_info['password'], acc_info['mail']])

Filled information:
User:		pmc210602194011
Password	-_Au8S908.RoI`5_}ZJ%
Mail		vigoxiy693@jmpant.com


In [11]:
ag.wd.quit()